## Loading Dataset

In [1]:
import pandas as pd 
import numpy as np 

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('../data/raw/occurrence.txt', sep='\t')
display(df.head(3))
df.shape

C:\Users\SABIO\AppData\Local\Temp\ipykernel_36260\3377765415.py:1: DtypeWarning: Columns (29,30,35,43,46,49,53,60,75,77,83,102,103,112,113,114,115,116,117,118,132,133,137,138,147,170,177) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/raw/occurrence.txt', sep='\t')


,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,institutionID,collectionID,datasetID,institutionCode,collectionCode,datasetName,ownerInstitutionCode,basisOfRecord,informationWithheld,dataGeneralizations,dynamicProperties,occurrenceID,catalogNumber,recordNumber,recordedBy,recordedByID,individualCount,organismQuantity,organismQuantityType,sex,lifeStage,reproductiveCondition,caste,behavior,vitality,establishmentMeans,degreeOfEstablishment,pathway,georeferenceVerificationStatus,occurrenceStatus,preparations,disposition,associatedOccurrences,associatedReferences,associatedSequences,associatedTaxa,otherCatalogNumbers,occurrenceRemarks,organismID,organismName,organismScope,associatedOrganisms,previousIdentifications,organismRemarks,materialEntityID,materialEntityRemarks,verbatimLabel,materialSampleID,eventID,parentEventID,eventType,fieldNumber,eventDate,eventTime,startDayOfYear,endDayOfYear,year,month,day,verbatimEventDate,habitat,samplingProtocol,sampleSizeValue,sampleSizeUnit,samplingEffort,fieldNotes,eventRemarks,locationID,higherGeographyID,higherGeography,continent,waterBody,islandGroup,island,countryCode,stateProvince,county,municipality,locality,verbatimLocality,verbatimElevation,verticalDatum,verbatimDepth,minimumDistanceAboveSurfaceInMeters,maximumDistanceAboveSurfaceInMeters,locationAccordingTo,locationRemarks,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,coordinatePrecision,pointRadiusSpatialFit,verbatimCoordinateSystem,verbatimSRS,footprintWKT,footprintSRS,footprintSpatialFit,georeferencedBy,georeferencedDate,georeferenceProtocol,georeferenceSources,georeferenceRemarks,geologicalContextID,earliestEonOrLowestEonothem,latestEonOrHighestEonothem,earliestEraOrLowestErathem,latestEraOrHighestErathem,earliestPeriodOrLowestSystem,latestPeriodOrHighestSystem,earliestEpochOrLowestSeries,latestEpochOrHighestSeries,earliestAgeOrLowestStage,latestAgeOrHighestStage,lowestBiostratigraphicZone,highestBiostratigraphicZone,lithostratigraphicTerms,group,formation,member,bed,identificationID,verbatimIdentification,identificationQualifier,typeStatus,identifiedBy,identifiedByID,dateIdentified,identificationReferences,identificationVerificationStatus,identificationRemarks,taxonID,scientificNameID,acceptedNameUsageID,parentNameUsageID,originalNameUsageID,nameAccordingToID,namePublishedInID,taxonConceptID,scientificName,acceptedNameUsage,parentNameUsage,originalNameUsage,nameAccordingTo,namePublishedIn,namePublishedInYear,higherClassification,kingdom,phylum,class,order,superfamily,family,subfamily,tribe,subtribe,genus,genericName,subgenus,infragenericEpithet,specificEpithet,infraspecificEpithet,cultivarEpithet,taxonRank,verbatimTaxonRank,vernacularName,nomenclaturalCode,taxonomicStatus,nomenclaturalStatus,taxonRemarks,datasetKey,publishingCountry,lastInterpreted,elevation,elevationAccuracy,depth,depthAccuracy,distanceFromCentroidInMeters,issue,mediaType,hasCoordinate,hasGeospatialIssues,taxonKey,acceptedTaxonKey,kingdomKey,phylumKey,classKey,orderKey,familyKey,genusKey,subgenusKey,speciesKey,species,acceptedScientificName,verbatimScientificName,typifiedName,protocol,lastParsed,lastCrawled,repatriated,relativeOrganismQuantity,projectId,isSequenced,gbifRegion,publishedByGbifRegion,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory
0,931073241,NaN,NaN,en,CC0_1_0,NaN,NaN,NaN,University of Guelph,NaN,NaN,NaN,http://dataset.canadensys.net/oac-specimens,Biodiversity Institute of Ontario,OAC,Ontario Agriculture College Herbarium (OAC-BIO),NaN,PRESERVED_SPECIMEN,NaN,NaN,NaN,41403,53588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PRESENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

(7240, 223)

## Extracting Meaninful Variables

Found some occurrances appearing in ASIA and CANADA. These should be removed.

In [3]:
display(df['continent'].value_counts())
display(df[df['verbatimLocality'].str.startswith('Canada', na=False)]['verbatimLocality'].value_counts())

continent
NORTH_AMERICA    6287
ASIA               15
Name: count, dtype: int64

After ASIA and CANADA have been removed, we select all variables related to location and time in order to further clean the dataset and geocode any locations with fragmented data.

In [6]:
# Create list of columns from the comment
columns_to_keep = ['gbifID','reproductiveCondition', 'occurrenceStatus', 
                  'eventDate', 'eventTime', 'startDayOfYear', 'endDayOfYear', 'year',
                  'month', 'day', 'verbatimEventDate', 'habitat', 'higherGeography',
                  'continent', 'countryCode', 'stateProvince', 'county', 'municipality',
                  'locality', 'verbatimLocality', 'verbatimElevation',
                  'minimumDistanceAboveSurfaceInMeters', 'maximumDistanceAboveSurfaceInMeters',
                  'decimalLatitude', 'decimalLongitude', 'coordinateUncertaintyInMeters',
                  'georeferencedDate', 'dateIdentified', 'elevation', 'elevationAccuracy',
                  'hasCoordinate', 'hasGeospatialIssues']

# Drop rows where continent is ASIA
df = df[
    (df['continent'] != 'ASIA') & 
    (~df['verbatimLocality'].str.startswith('Canada', na=False))
]

# Create a copy with only the specified columns
df_detailed = df[columns_to_keep].copy()

# Split columns into coordinate and location groups
coordinate_columns = ['decimalLatitude', 'decimalLongitude']
location_columns = ['countryCode', 'stateProvince', 'county', 'municipality', 'locality']

# Keep rows that have either:
# 1. Both coordinates (lat & long)
# 2. At least one location field filled
has_coordinates = df_detailed[coordinate_columns].notna().all(axis=1)
has_location = df_detailed[location_columns].notna().any(axis=1)

# Apply the filter to keep rows with either coordinates or location info
df_pruned = df_detailed[has_coordinates | has_location].copy()

print(f"\nOriginal shape: {df_detailed.shape}")
print(f"Shape after filtering: {df_pruned.shape}")
print(f"Removed {df_detailed.shape[0] - df_pruned.shape[0]} rows")

# Save pruned dataset
#df_pruned.to_csv('../data/processed/pruned.csv', index=False)

# Display first few rows to verify the data
print("\nPruned dataset preview:")
# Display removed rows
removed_rows = df_detailed[~(has_coordinates | has_location)]
print("\nRemoved rows preview:")
display(removed_rows.head())

print("\nPruned dataset preview:")
display(df_pruned.head())


Original shape: (7046, 32)
Shape after filtering: (6823, 32)
Removed 223 rows

Pruned dataset preview:

Removed rows preview:


,gbifID,reproductiveCondition,occurrenceStatus,eventDate,eventTime,startDayOfYear,endDayOfYear,year,month,day,verbatimEventDate,habitat,higherGeography,continent,countryCode,stateProvince,county,municipality,locality,verbatimLocality,verbatimElevation,minimumDistanceAboveSurfaceInMeters,maximumDistanceAboveSurfaceInMeters,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,georeferencedDate,dateIdentified,elevation,elevationAccuracy,hasCoordinate,hasGeospatialIssues
0,931073241,NaN,PRESENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
1,931040138,NaN,PRESENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
19,695167951,NaN,PRESENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
20,695167945,NaN,PRESENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
21,694280617,NaN,PRESENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False



Pruned dataset preview:


,gbifID,reproductiveCondition,occurrenceStatus,eventDate,eventTime,startDayOfYear,endDayOfYear,year,month,day,verbatimEventDate,habitat,higherGeography,continent,countryCode,stateProvince,county,municipality,locality,verbatimLocality,verbatimElevation,minimumDistanceAboveSurfaceInMeters,maximumDistanceAboveSurfaceInMeters,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,georeferencedDate,dateIdentified,elevation,elevationAccuracy,hasCoordinate,hasGeospatialIssues
2,894988018,NaN,PRESENT,1970-07-11,NaN,192.0,192.0,1970.0,7.0,11.0,NaN,"scree slope, facing east",NaN,NORTH_AMERICA,CA,Alberta,NaN,Waterton Lakes National Park,West of Lost Lake,NaN,7000 feet,NaN,NaN,49.133333,-114.15,NaN,NaN,NaN,2133.0,0.0,True,False
3,894988016,NaN,PRESENT,1968-08-04,NaN,217.0,217.0,1968.0,8.0,4.0,NaN,Trail side,NaN,NORTH_AMERICA,CA,Alberta,NaN,Waterton Lakes National Park,"near campsite, north-west of Cameron Lake",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
4,894987984,NaN,PRESENT,1970-07-11,NaN,192.0,192.0,1970.0,7.0,11.0,NaN,"scree slope, facing east",NaN,NORTH_AMERICA,CA,Alberta,NaN,Waterton Lakes National Park,West of Lost Lake,NaN,7000 feet,NaN,NaN,49.133333,-114.15,NaN,NaN,NaN,2133.0,0.0,True,False
5,894987983,NaN,PRESENT,1959-07-10,NaN,191.0,191.0,1959.0,7.0,10.0,NaN,Coniferous forest,NaN,NORTH_AMERICA,US,Wyoming,NaN,NaN,Teton Pass,NaN,8400 feet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2560.0,0.0,False,False
6,894987960,NaN,PRESENT,1970-07-11,NaN,192.0,192.0,1970.0,7.0,11.0,NaN,"scree slope, facing east",NaN,NORTH_AMERICA,CA,Alberta,NaN,Waterton Lakes National Park,West of Lost Lake,NaN,7000 feet,NaN,NaN,49.133333,-114.15,NaN,NaN,NaN,2133.0,0.0,True,False


## Geocoding (Lon & Lat Cleaning)

858 rows that do not have longitude or latitude however might have relevant data for geocoding including their locality.

In [7]:
df_pruned[df_pruned['decimalLatitude'].isna() & df_pruned['decimalLongitude'].isna()].shape

(858, 32)

In [ ]:
import sys
sys.path.append('..')
from src.geocoding import process_dataset
import pandas as pd
import os

# Load your original dataset (assuming it's df_pruned or similar)
# Replace 'df_pruned' with whatever your original dataframe variable name is
df_original = df_pruned.copy()  # or whatever your original dataframe is called

# Process ALL rows that don't have coordinates with the updated logic
df_enhanced, stats = process_dataset(df_original)

# Save the updated dataframe
output_path = '../data/processed/updated_coordinates_with_verbatim.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df_enhanced.to_csv(output_path, index=False)
print(f"\nSaved updated coordinates to: {output_path}")

# Show remaining rows without coordinates
print("\nRemaining rows without coordinates:")
display(df_enhanced[df_enhanced['decimalLatitude'].isna() & df_enhanced['decimalLongitude'].isna()])

# Show rows where LLM was used
print("\nRows where LLM was used for geocoding:")
display(df_enhanced[df_enhanced['used_llm']])

# Show some examples of verbatimLocality usage
print("\nExamples of verbatimLocality data:")
verbatim_examples = df_enhanced[df_enhanced['verbatimLocality'].notna()][['locality', 'verbatimLocality']].head(10)
display(verbatim_examples)

# Show cases where locality is empty but verbatimLocality has data
print("\nCases where locality is empty but verbatimLocality has data:")
empty_locality_with_verbatim = df_enhanced[
    (df_enhanced['locality'].isna() | 
     df_enhanced['locality'].str.lower().isin(['no specific locality recorded', 'no locality recorded', 'nan', 'none', ''])) &
    df_enhanced['verbatimLocality'].notna() &
    (~df_enhanced['verbatimLocality'].str.lower().isin(['no verbatim locality recorded', 'no locality recorded', 'nan', 'none', '']))
][['locality', 'verbatimLocality']].head(10)
display(empty_locality_with_verbatim)

# Show statistics
print(f"\nFinal Statistics:")
print(f"Total rows processed: {stats['total_rows']}")
print(f"Rows with missing coordinates: {stats['missing_coords']}")
print(f"Successfully updated: {stats['successfully_updated']}")
print(f"Failed updates: {stats['failed_updates']}")
print(f"Rows where LLM was used: {stats['llm_used_count']}")
success_rate = (stats['successfully_updated'] / stats['missing_coords'] * 100)
print(f"Success rate: {success_rate:.2f}%")

Geocoding progress: 100%|██████████| 858/858 [34:59<00:00,  2.45s/it]


Geocoding Statistics:
Total rows processed: 6823
Rows with missing coordinates: 858
Successfully updated: 773
Failed updates: 85
Rows where LLM was used: 86
Success rate: 90.09%

Detailed logs saved to: c:\Users\SABIO\Documents\GitHub\Capstone-Microsoft\notebooks\..\logs\geocoding_20250618_175606.log

Saved updated coordinates to: ../data/processed/updated_coordinates_with_verbatim_2.csv

Remaining rows without coordinates:


,gbifID,reproductiveCondition,occurrenceStatus,eventDate,eventTime,startDayOfYear,endDayOfYear,year,month,day,verbatimEventDate,habitat,higherGeography,continent,countryCode,stateProvince,county,municipality,locality,verbatimLocality,verbatimElevation,minimumDistanceAboveSurfaceInMeters,maximumDistanceAboveSurfaceInMeters,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,georeferencedDate,dateIdentified,elevation,elevationAccuracy,hasCoordinate,hasGeospatialIssues,used_llm
30,5174618020,NaN,PRESENT,1800-01-01/2008-04-25,1800-01-01/2008-04-25,1.0,116.0,NaN,NaN,NaN,not recorded,NaN,no higher geography recorded,NaN,NaN,NaN,NaN,NaN,No specific locality recorded.,No verbatim locality recorded,NaN,NaN,NaN,NaN,NaN,NaN,2012-07-08 00:00:00,2008-04-25T00:00:00,NaN,NaN,False,False,False
1836,441858009,NaN,PRESENT,1967-07-26,NaN,207.0,207.0,1967.0,7.0,26.0,NaN,NaN,NaN,NORTH_AMERICA,CA,British Columbia,NaN,NaN,Aeta Lake.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
1849,441857874,NaN,PRESENT,1925-08,NaN,NaN,NaN,1925.0,8.0,NaN,NaN,NaN,NaN,NaN,US,Washington,NaN,NaN,"Inter Fork of White River, Mount Rainier.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1944-01-01T00:00:00,1524.0,0.0,False,False,False
1865,441857743,NaN,PRESENT,1938-06-20,NaN,171.0,171.0,1938.0,6.0,20.0,NaN,NaN,NaN,NORTH_AMERICA,CA,British Columbia,NaN,NaN,"Near Lake Bootahnie, Marble Mountains.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1524.0,0.0,False,False,False
1885,441857716,NaN,PRESENT,1939-06-17,NaN,168.0,168.0,1939.0,6.0,17.0,NaN,NaN,NaN,NORTH_AMERICA,CA,British Columbia,NaN,NaN,"Sumup Cabin, southwest of Mount Baldy, 9 miles...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1326.0,46.0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6338,1302601879,NaN,PRESENT,1983,NaN,NaN,NaN,1983.0,NaN,NaN,NaN,NaN,NaN,NaN,US,NaN,NaN,NaN,"+9 miles southeast of McKenzie Bridge, Willame...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
6373,1302588522,NaN,PRESENT,1992-07-22,NaN,204.0,204.0,1992.0,7.0,22.0,NaN,NaN,NaN,NaN,US,California,NaN,NaN,"Near summit of o''brien-happy camp road, at ju...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
6466,1302588421,NaN,PRESENT,1892-07-06,NaN,188.0,188.0,1892.0,7.0,6.0,NaN,NaN,NaN,NaN,US,Idaho,NaN,NaN,Owyhee Mountains,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
6675,1302583044,NaN,PRESENT,1892-06-20,NaN,172.0,172.0,1892.0,6.0,20.0,NaN,NaN,NaN,NaN,US,Washington,NaN,NaN,[no locality given on the label].,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False



Rows where LLM was used for geocoding:


,gbifID,reproductiveCondition,occurrenceStatus,eventDate,eventTime,startDayOfYear,endDayOfYear,year,month,day,verbatimEventDate,habitat,higherGeography,continent,countryCode,stateProvince,county,municipality,locality,verbatimLocality,verbatimElevation,minimumDistanceAboveSurfaceInMeters,maximumDistanceAboveSurfaceInMeters,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,georeferencedDate,dateIdentified,elevation,elevationAccuracy,hasCoordinate,hasGeospatialIssues,used_llm
3,894988016,NaN,PRESENT,1968-08-04,NaN,217.0,217.0,1968.0,8.0,4.0,NaN,Trail side,NaN,NORTH_AMERICA,CA,Alberta,NaN,Waterton Lakes National Park,"near campsite, north-west of Cameron Lake",NaN,NaN,NaN,NaN,44.553159,-78.761708,NaN,NaN,NaN,NaN,NaN,False,False,True
29,5174673580,NaN,PRESENT,1976-07-16,1976-07-16,198.0,198.0,1976.0,7.0,16.0,16 July 1976,NaN,"United States, Washington",NaN,US,Washington,NaN,NaN,"Chelan Co., Mazama: Along rocky shore of Lake ...","Chelan Co., Mazama: Along rocky shore of Lake ...",NaN,NaN,NaN,48.735298,-121.864987,NaN,1800-01-01 00:00:00,1800-01-01T00:00:00,NaN,NaN,False,False,True
67,5089841149,NaN,PRESENT,1925-07-25T00:00,NaN,206.0,206.0,1925.0,7.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Seeley Lake, west side slough, north end",NaN,NaN,NaN,NaN,47.155299,-113.448428,NaN,NaN,NaN,NaN,NaN,False,False,True
1833,441858030,NaN,PRESENT,1941-07-08,NaN,189.0,189.0,1941.0,7.0,8.0,NaN,NaN,NaN,NORTH_AMERICA,CA,British Columbia,NaN,NaN,West side of Mountain Revelstoke.,NaN,NaN,NaN,NaN,50.969263,-118.109209,NaN,NaN,2006-04-06T00:00:00,1616.0,0.0,False,False,True
1851,441857872,NaN,PRESENT,1907-08-02,NaN,214.0,214.0,1907.0,8.0,2.0,NaN,NaN,NaN,NaN,US,Washington,NaN,NaN,Olympic Mountains. Elwha Valley.,NaN,NaN,NaN,NaN,47.703130,-123.522704,NaN,NaN,NaN,915.0,305.0,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6972,1302581773,NaN,PRESENT,1896-06-19,NaN,171.0,171.0,1896.0,6.0,19.0,NaN,NaN,NaN,NaN,US,Washington,NaN,NaN,"Cascade Mountains, Mount Rainier and adjacent ...",NaN,NaN,NaN,NaN,49.000491,-119.755842,NaN,NaN,NaN,NaN,NaN,False,False,True
7013,1302579961,NaN,PRESENT,1923-07-02,NaN,183.0,183.0,1923.0,7.0,2.0,NaN,NaN,NaN,NaN,US,Montana,NaN,NaN,Missoula National Forest: Montour Creek.,NaN,NaN,NaN,NaN,47.060103,-115.115845,NaN,NaN,NaN,NaN,NaN,False,False,True
7152,1302577454,NaN,PRESENT,1942-06-10,NaN,161.0,161.0,1942.0,6.0,10.0,NaN,NaN,NaN,NaN,US,Montana,NaN,NaN,"Glacier National Park, south slope of Quartz R...",NaN,NaN,NaN,NaN,48.617486,-113.760885,NaN,NaN,NaN,NaN,NaN,False,False,True
7191,1302577415,NaN,PRESENT,1938-05-14,NaN,134.0,134.0,1938.0,5.0,14.0,NaN,NaN,NaN,NaN,US,Idaho,NaN,NaN,Near South shore of Priest Lake.,NaN,NaN,NaN,NaN,48.609535,-116.878512,NaN,NaN,NaN,NaN,NaN,False,False,True



Examples of verbatimLocality data:


,locality,verbatimLocality
10,Mount Edith Cavell,"Jasper Nat. Park, Mt. Edith Cavell: up from ch..."
11,Ewin pass,"Flora of the Elk Valley, B.C. Ewin Pass - Mnt...."
12,Nose Mountain,Nose Mt. 64-11-W6. Conifer forest (Ab. la. wit...
13,Kakwa Falls,Kakwa Wildland Provincial Park: Kakwa Falls Tr...
14,Valleyview,Alberta: 17.6 km east of Valleyview with acces...
15,Fox Creek,19.5 km on hwy 947 (south of hwy #43); 11.2 km...
16,Fox Creek,Southeast of Fox Creek on road 947. 29.9 km S...
17,Valleyview,Alberta: 17.6 km east of Valleyview with acces...
18,Hinton,Hinton Area: 1.0 mi. south of Hwy. 16 on Hwy....
24,NaN,"Okanagan Mountain Provincial Park, BC, Canada"



Cases where locality is empty but verbatimLocality has data:


,locality,verbatimLocality
24,NaN,"Okanagan Mountain Provincial Park, BC, Canada"
25,NaN,"Okanagan Mountain Provincial Park, BC, Canada"
26,NaN,"Anthill Trail, Hood River County, OR, USA"
27,NaN,"Fraser Valley, CA-BC, CA"
28,NaN,"Eastern Hillsides, Chilliwack, BC, CA"
32,NaN,"Olympic National Forest, Hoodsport, WA, US"
33,NaN,"Okanogan County, WA, USA"
34,NaN,"Allouez, MI 49805, USA"
35,NaN,"Clallam County, Olympic, Olympic National Park..."
36,NaN,"King, Washington, United States"



Final Statistics:
Total rows processed: 6823
Rows with missing coordinates: 858
Successfully updated: 773
Failed updates: 85
Rows where LLM was used: 86
Success rate: 90.09%


## Manual Geocoding 

There are not many coordinates left and considering we tried many different options for the rest of the dataset, we can either drop it or manually geocode the remaining values using the locality / municipality.

In [6]:
import pandas as pd
pd.set_option('display.max_columns', None)
verb = pd.read_csv('../data/processed/updated_coordinates_with_verbatim.csv')
verb[verb['decimalLatitude'].isna() & verb['decimalLongitude'].isna()].head(3)

,gbifID,reproductiveCondition,occurrenceStatus,eventDate,eventTime,startDayOfYear,endDayOfYear,year,month,day,verbatimEventDate,habitat,higherGeography,continent,countryCode,stateProvince,county,municipality,locality,verbatimLocality,verbatimElevation,minimumDistanceAboveSurfaceInMeters,maximumDistanceAboveSurfaceInMeters,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,georeferencedDate,dateIdentified,elevation,elevationAccuracy,hasCoordinate,hasGeospatialIssues,used_llm
24,5174618020,NaN,PRESENT,1800-01-01/2008-04-25,1800-01-01/2008-04-25,1.0,116.0,NaN,NaN,NaN,not recorded,NaN,no higher geography recorded,NaN,NaN,NaN,NaN,NaN,No specific locality recorded.,No verbatim locality recorded,NaN,NaN,NaN,NaN,NaN,NaN,2012-07-08 00:00:00,2008-04-25T00:00:00,NaN,NaN,False,False,False
1829,441858009,NaN,PRESENT,1967-07-26,NaN,207.0,207.0,1967.0,7.0,26.0,NaN,NaN,NaN,NORTH_AMERICA,CA,British Columbia,NaN,NaN,Aeta Lake.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
1842,441857874,NaN,PRESENT,1925-08,NaN,NaN,NaN,1925.0,8.0,NaN,NaN,NaN,NaN,NaN,US,Washington,NaN,NaN,"Inter Fork of White River, Mount Rainier.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1944-01-01T00:00:00,1524.0,0.0,False,False,False


In [7]:
verb[verb['decimalLatitude'].isna() & verb['decimalLongitude'].isna()]['locality'].value_counts()

locality
Copenhagen University's Arboretum                                                                       5
Stevens Pass Region, Cascade Mountains.                                                                 2
[no locality given on the label].                                                                       2
Owyhee Mountains                                                                                        2
Near Lake Bootahnie, Marble Mountains.                                                                  1
Inter Fork of White River, Mount Rainier.                                                               1
Aeta Lake.                                                                                              1
No specific locality recorded.                                                                          1
Sumup Cabin, southwest of Mount Baldy, 9 miles northwest of Ross Spur.                                  1
4 mi. E of Stevens Pass on U.S. 2 by 

In [8]:
manual_geocodes = {
    "Copenhagen University's Arboretum": (55.7847, 12.5197),  # Charlottenlund, Denmark
    "Stevens Pass Region, Cascade Mountains.": (47.7458, -121.0896),
    "Owyhee Mountains": (42.6197, -116.7894),
    "Near South shore of Priest Lake.": (48.5272, -116.9150),
    "Near Lake Bootahnie, Marble Mountains.": (41.6528, -123.7283),
    "Aeta Lake.": (49.7683, -125.0020),
    "Sumup Cabin, southwest of Mount Baldy, 9 miles northwest of Ross Spur.": (48.3039, -117.2667),
    "String Lake, Teton Park.": (43.8039, -110.7456),
    "Grimes Creek, Boise National Forest.": (43.6750, -115.8947),
    '"Sphinx glacier foreland, Garbildi park Locality Glacial moraines"': (49.9615, -123.0135),
    '"11 mi. S of Lost Trail Pass on U.S. 93 N of North Fork at Twin Crk. Campgrd., Salmon Nat 1 Forest."': (44.9461, -113.9361),
    "Cascade Mtns": (47.5000, -121.8000),
    "Near summit of the Metolias Trail, Cascade Mts.": (44.4608, -121.8053),
    "Blewett Pass Region. Cascade Mountains.": (47.3333, -120.6333),
    "Deep woods, Selkirk Mts.": (49.0000, -117.0000),
    "Chelan County, Okanogan National Forest. Rainy Lake Outlet": (48.5167, -120.7500),
    "near West Thumb. Yellowstone Natl Park.": (44.4170, -110.5730),
    "Meagher Co: in lodgepole forest. Harley Park, ca. 6 mi. west of Neihart, Little Belt Mts.": (46.9581, -110.8262),
    "Steven's Pacific Rairoad Survey": (47.0000, -120.0000),  # Approx. WA Cascades
    "East side of Warm River Canyon, near Henry's Fork of Snake River.": (44.0713, -111.2935),
    "Oregon Boundry Commission": (44.0000, -120.5000),  # Approx. central Oregon
    "Headwaters of Jocko River": (47.2000, -114.0000),
    "+9 miles southeast of McKenzie Bridge, Willamette National Forest.": (44.0935, -122.0135),
    "Rogue river valley.": (42.4331, -123.3286),
    "Inter Fork of White River, Mount Rainier.": (46.8535, -121.7017),
    "4 mi. E of Stevens Pass on U.S. 2 by turnoff in Spruce-Hemlock forest & alpine.": (47.7440, -120.9840),
    "Near summit of o''brien-happy camp road, at junction of bolan lake road, del norte County.": (41.8989, -123.6705)
}

manual_geocodes_municipality = {
    "Green Mountain, near Haylmore.": (50.6983, -122.4821),
    "Rogue river valley.": (42.4331, -123.3286),
    "Mt. hood, wapinilia pass.": (45.1600, -121.7810),
    "Above the government camp, Mt. hood.": (45.3106, -121.7526),
    "Blue Mts.": (45.7167, -118.1000),
    "Cameron lake, Waterton Lakes National Park.": (49.0239, -113.8982),
    "Slide overlooking Meager Creek, ca. 1 mile above its junction with Lillooet River.": (50.4675, -123.4753),
    "Mile 355, Alaska highway.": (60.0954, -128.7983),
    "Mile 195, Alaska highway.": (59.3769, -126.9608),
    "10 miles north of Summit Lake on Hart Highway.": (55.3033, -122.6625),
    "Arrow park, fisher creek.": (50.3210, -117.8820),
    "One mile N of parsnip river along the hart highway.": (55.6833, -122.5833),
    "Takla Lake about 9 miles northwest of Takla Landing.": (55.7660, -126.2500),
    "Olympic Mountains. Elwha Valley.": (47.9714, -123.6061),
    "Along trail to forest fire lookout station, mcbride.": (53.3072, -120.1750),
    "Arrow park, stevens creek.": (50.3070, -117.8700),
    "Arrow park, makison flats.": (50.3000, -117.8000),
    "Dam Mt.": (50.0719, -123.0833),
    "Stevens Pass Region, Cascade Mountains.": (47.7458, -121.0896),
    "Owyhee Mountains": (42.6197, -116.7894),
    "Echo Lake, Rocky Mountains, Flathead National Forest.": (47.5441, -113.9427),
    "Inter Fork of White River, Mount Rainier.": (46.8535, -121.7017),
    "Cascade Mountains, Mount Rainier and adjacent mountains.": (46.8523, -121.7603),
    "Near hope.": (49.3824, -121.4409),
    "Burman Lake, drainage, Burman River. Vancouver Island.": (49.6887, -126.0917),
    "Trail that runs along east shore of burges pass, emerald lake.": (51.4206, -116.4728),
    "1.7 miles south of Lakeview.": (41.9900, -120.5581),
    "Aeta Lake.": (49.7683, -125.0020),
    "Burges pass.": (51.4220, -116.4850),
    "Trail to Elk Summit to Kooskooska. Selway Forest. (county unknown)": (46.1476, -114.7625),
    "Glacier National Park, Sun Camp.": (48.6101, -113.4541),
    "Between horsefly & crooked lake.": (52.2667, -121.1167),
    "Missoula County, north of upper Holland Lake, swan range, ca. 80 miles NE of Missoula, township 30 n, r15w, s. 29.": (47.4167, -113.4167),
    "Mt. richards, NE of 60 degree slope, Waterton Lakes National Park.": (49.1167, -113.7833),
    "Azouzetta Lake on Hart Highway.": (55.3667, -122.7167),
    "Seven Devils Mountains Nez Perce National Forest.": (45.2506, -116.6706),
    "Ant Basin, Idaho, East Lago.": (42.1722, -111.3833),
    "Near South shore of Priest Lake.": (48.5272, -116.9150),
    "Sumup Cabin, southwest of Mount Baldy, 9 miles northwest of Ross Spur.": (48.3039, -117.2667),
    "Monashee road above inooklin creek, just SE of banting creek.": (50.2375, -118.2428),
    "Near kootenay florence mine, ainsworth.": (49.7589, -116.9256),
    "West side of Mountain Revelstoke.": (51.0312, -118.1790),
    "Along Hope-Princeton Highway where road crosses Skaist River.": (49.3000, -120.7833),
    "Circa. 4.25 miles northeast by east of Falkland.": (50.5383, -119.1667),
    "East of takatoot l.": (54.3167, -126.2500),
    "Shore of Blackwater Lake, circa 32 miles north of Golden.": (51.7833, -116.9667),
    "Outpost lake trail, Jasper National Park.": (52.8442, -117.9394),
    "On Bolean Lake Trail, circa. 4 miles northeast by east of Falkland.": (50.5400, -119.1500),
    "Glacier National Park, south slope of Quartz Ridge, north of Quartz Lake.": (48.8667, -114.0000),
    "Headwaters of Jocko River": (47.2000, -114.0000),
    "Near Lake Bootahnie, Marble Mountains.": (41.6528, -123.7283),
    "Near south shore of Priest Lake.": (48.5272, -116.9150),
}

def fill_manual_coords(row):
    locality = row['locality']
    if pd.isna(row['decimalLatitude']) and pd.isna(row['decimalLongitude']) and locality in manual_geocodes:
        lat, lon = manual_geocodes[locality]
        row['decimalLatitude'] = lat
        row['decimalLongitude'] = lon
    return row

def fill_from_municipality(row):
    if pd.isna(row['decimalLatitude']) and pd.isna(row['decimalLongitude']):
        muni = row['municipality']
        if pd.notna(muni) and muni in manual_geocodes_municipality:
            lat, lon = manual_geocodes_municipality[muni]
            row['decimalLatitude'] = lat
            row['decimalLongitude'] = lon
    return row

verb = verb.apply(fill_manual_coords, axis=1)
display(verb[verb['decimalLatitude'].isna() & verb['decimalLongitude'].isna()].shape)
display(verb[verb['decimalLatitude'].isna() & verb['decimalLongitude'].isna()]['locality'].value_counts())

verb = verb.apply(fill_from_municipality, axis=1)
display(verb[verb['decimalLatitude'].isna() & verb['decimalLongitude'].isna()].shape)
display(verb[verb['decimalLatitude'].isna() & verb['decimalLongitude'].isna()]['municipality'].value_counts())

verb[verb['decimalLatitude'].isna() & verb['decimalLongitude'].isna()].head()

(58, 33)

locality
[no locality given on the label].    2
No specific locality recorded.       1
[no additional data]                 1
Name: count, dtype: int64

(5, 33)

municipality
[no locality given on the label].    1
Name: count, dtype: int64

,gbifID,reproductiveCondition,occurrenceStatus,eventDate,eventTime,startDayOfYear,endDayOfYear,year,month,day,verbatimEventDate,habitat,higherGeography,continent,countryCode,stateProvince,county,municipality,locality,verbatimLocality,verbatimElevation,minimumDistanceAboveSurfaceInMeters,maximumDistanceAboveSurfaceInMeters,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,georeferencedDate,dateIdentified,elevation,elevationAccuracy,hasCoordinate,hasGeospatialIssues,used_llm
24,5174618020,NaN,PRESENT,1800-01-01/2008-04-25,1800-01-01/2008-04-25,1.0,116.0,NaN,NaN,NaN,not recorded,NaN,no higher geography recorded,NaN,NaN,NaN,NaN,NaN,No specific locality recorded.,No verbatim locality recorded,NaN,NaN,NaN,NaN,NaN,NaN,2012-07-08 00:00:00,2008-04-25T00:00:00,NaN,NaN,False,False,False
1958,441857473,NaN,PRESENT,1892-06-20,NaN,172.0,172.0,1892.0,6.0,20.0,NaN,NaN,NaN,NaN,US,Washington,NaN,NaN,[no locality given on the label].,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1944-01-01T00:00:00,NaN,NaN,False,False,False
5000,1304428407,NaN,PRESENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,California,NaN,NaN,[no additional data],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
5291,1302779510,NaN,PRESENT,1892-06-20,NaN,172.0,172.0,1892.0,6.0,20.0,NaN,NaN,NaN,NaN,US,Washington,NaN,[no locality given on the label].,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
6258,1302583044,NaN,PRESENT,1892-06-20,NaN,172.0,172.0,1892.0,6.0,20.0,NaN,NaN,NaN,NaN,US,Washington,NaN,NaN,[no locality given on the label].,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False


Last couple rows have no locality, verbatimLocality or municipality to work with and therefore we will drop these aswell.

In [9]:
verb = verb.dropna(subset=['decimalLatitude', 'decimalLongitude'])  # Final clean-up
verb[verb['decimalLatitude'].isna() & verb['decimalLongitude'].isna()].shape

(0, 33)

Get rid of all the columns we used and clean up our dataset. We will keep the date and time information as this will be cleaned next in a similar fashion

In [11]:
# Create a subset with just the columns we need for coordinates
columns_to_keep = [col for col in verb.columns if col not in [
    'higherGeography', 'continent', 'countryCode', 'stateProvince',
    'county', 'municipality', 'locality', 'verbatimLocality', 
    'hasCoordinate', 'used_llm', 'coordinateUncertaintyInMeters', "hasGeospatialIssues"
]]

coords_subset = verb[columns_to_keep]
coords_subset.head(3)

,gbifID,reproductiveCondition,occurrenceStatus,eventDate,eventTime,startDayOfYear,endDayOfYear,year,month,day,verbatimEventDate,habitat,verbatimElevation,minimumDistanceAboveSurfaceInMeters,maximumDistanceAboveSurfaceInMeters,decimalLatitude,decimalLongitude,georeferencedDate,dateIdentified,elevation,elevationAccuracy
0,894988018,NaN,PRESENT,1970-07-11,NaN,192.0,192.0,1970.0,7.0,11.0,NaN,"scree slope, facing east",7000 feet,NaN,NaN,49.133333,-114.150000,NaN,NaN,2133.0,0.0
1,894988016,NaN,PRESENT,1968-08-04,NaN,217.0,217.0,1968.0,8.0,4.0,NaN,Trail side,NaN,NaN,NaN,44.553159,-78.761708,NaN,NaN,NaN,NaN
2,894987984,NaN,PRESENT,1970-07-11,NaN,192.0,192.0,1970.0,7.0,11.0,NaN,"scree slope, facing east",7000 feet,NaN,NaN,49.133333,-114.150000,NaN,NaN,2133.0,0.0


In [13]:
coords_subset.shape

(6818, 21)

In [23]:
coords_subset.to_csv('../data/processed/coordinates_cleaned.csv', index=False)

## Datetime Cleaning

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
dat = pd.read_csv('../data/processed/coordinates_cleaned.csv')
display(dat.columns)
dat[dat['eventDate'].isna()].shape

Index(['gbifID', 'reproductiveCondition', 'occurrenceStatus', 'eventDate',
       'eventTime', 'startDayOfYear', 'endDayOfYear', 'year', 'month', 'day',
       'verbatimEventDate', 'habitat', 'verbatimElevation',
       'minimumDistanceAboveSurfaceInMeters',
       'maximumDistanceAboveSurfaceInMeters', 'decimalLatitude',
       'decimalLongitude', 'georeferencedDate', 'dateIdentified', 'elevation',
       'elevationAccuracy'],
      dtype='object')

(170, 21)

In [2]:
# Create a copy of the dataframe to work with
test = dat.copy()

# First try to convert eventDate directly to datetime where possible
test['datetime'] = pd.to_datetime(test['eventDate'], errors='coerce')

# For rows where eventDate conversion failed, try to construct datetime from year/month/day
mask = test['datetime'].isna()
test.loc[mask, 'datetime'] = pd.to_datetime({
    'year': test.loc[mask, 'year'],
    'month': test.loc[mask, 'month'], 
    'day': test.loc[mask, 'day']
}, errors='coerce')

# For rows still missing datetime, try to use startDayOfYear with year
mask = test['datetime'].isna()
def day_of_year_to_date(row):
    try:
        if pd.notna(row['year']) and pd.notna(row['startDayOfYear']):
            return pd.Timestamp(year=int(row['year']), month=1, day=1) + pd.Timedelta(days=int(row['startDayOfYear'])-1)
    except:
        return pd.NaT
    return pd.NaT

test.loc[mask, 'datetime'] = test[mask].apply(day_of_year_to_date, axis=1)

# For remaining NaN, try verbatimEventDate
mask = test['datetime'].isna()
test.loc[mask, 'datetime'] = pd.to_datetime(test.loc[mask, 'verbatimEventDate'], errors='coerce')

# As last resort, use dateIdentified 
mask = test['datetime'].isna()
test.loc[mask, 'datetime'] = pd.to_datetime(test.loc[mask, 'dateIdentified'], errors='coerce')

# Drop the original date/time columns since we now have a clean datetime column
test = test.drop(['eventDate', 'eventTime', 'startDayOfYear', 'endDayOfYear', 
                  'year', 'month', 'day', 'verbatimEventDate', 'dateIdentified'], axis=1)

test[test['datetime'].isna()].shape

C:\Users\SABIO\AppData\Local\Temp\ipykernel_28984\1460910216.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  test.loc[mask, 'datetime'] = pd.to_datetime(test.loc[mask, 'verbatimEventDate'], errors='coerce')


(543, 13)

In [3]:
dat = test.dropna(subset=['datetime'])  # Final clean-up
dat[dat['datetime'].isna()].shape

(0, 13)

In [4]:
dat = dat[['gbifID', 'datetime', 'decimalLatitude', 'decimalLongitude']]

In [5]:
dat.shape

(6275, 4)

In [6]:
dat.to_csv('../data/processed/date_location_occurrances.csv', index=False)

## Merging Datasets

In [7]:
import pandas as pd
huckleberry = pd.read_csv('../data/processed/date_location_occurrances.csv')
# huckleberry = huckleberry[['gbifID', 'datetime', 'decimalLatitude', 'decimalLongitude']]
display(huckleberry.head(3))
display(huckleberry.shape)

,gbifID,datetime,decimalLatitude,decimalLongitude
0,894988018,1970-07-11,49.133333,-114.150000
1,894988016,1968-08-04,44.553159,-78.761708
2,894987984,1970-07-11,49.133333,-114.150000


(6275, 4)

In [1]:
import pystac_client
import planetary_computer
import xarray as xr

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

asset = catalog.get_collection("gridmet").assets["zarr-abfs"]

ds = xr.open_zarr(
    asset.href,
    storage_options=asset.extra_fields["xarray:storage_options"],
    **asset.extra_fields["xarray:open_kwargs"]
)

In [2]:
ds

<xarray.Dataset> Size: 597GB
Dimensions:                                    (time: 15341, lat: 585,
                                                lon: 1386, crs: 1)
Coordinates:
  * crs                                        (crs) uint16 2B 3
  * lat                                        (lat) float64 5kB 49.4 ... 25.07
  * lon                                        (lon) float64 11kB -124.8 ... ...
  * time                                       (time) datetime64[ns] 123kB 19...
Data variables:
    air_temperature                            (time, lat, lon) float32 50GB dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    burning_index_g                            (time, lat, lon) float32 50GB dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    dead_fuel_moisture_1000hr                  (time, lat, lon) float32 50GB dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    dead_fuel_moisture_100hr                   (time, lat, lon) float32 50GB dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    mean_vapor_pressure_deficit                (time, lat, lon) float32 50GB dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    potential_evapotranspiration               (time, lat, lon) float32 50GB dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    precipitation_amount                       (time, lat, lon) float32 50GB dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    relative_humidity                          (time, lat, lon) float32 50GB dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    specific_humidity                          (time, lat, lon) float32 50GB dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    surface_downwelling_shortwave_flux_in_air  (time, lat, lon) float32 50GB dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    wind_from_direction                        (time, lat, lon) float32 50GB dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
    wind_speed                                 (time, lat, lon) float32 50GB dask.array<chunksize=(30, 585, 1386), meta=np.ndarray>
Attributes: (12/19)
    Conventions:                CF-1.6
    author:                     John Abatzoglou - University of Idaho, jabatz...
    coordinate_system:          EPSG:4326
    date:                       02 July 2019
    geospatial_bounds:          POLYGON((-124.7666666333333 49.40000000000000...
    geospatial_bounds_crs:      EPSG:4326
    ...                         ...
    geospatial_lon_units:       decimal_degrees east
    note1:                      The projection information for this file is: ...
    note2:                      Citation: Abatzoglou, J.T., 2013, Development...
    note3:                      Data in slices after last_permanent_slice (1-...
    note4:                      Data in slices after last_provisional_slice (...
    note5:                      Days correspond approximately to calendar day...

In [ ]:
import pandas as pd
from datetime import timedelta
from dask import delayed, compute
import xarray as xr
import pystac_client
import planetary_computer

# ----------------------------
# Load GBIF cleaned dataset
# ----------------------------
df = pd.read_csv("../data/processed/date_location_occurrances.csv")
df["datetime"] = pd.to_datetime(df["datetime"])

# # ----------------------------
# # Load GridMET via Planetary Computer
# # ----------------------------
# catalog = pystac_client.Client.open(
#     "https://planetarycomputer.microsoft.com/api/stac/v1",
#     modifier=planetary_computer.sign_inplace,
# )

# asset = catalog.get_collection("gridmet").assets["zarr-abfs"]
# ds = xr.open_zarr(
#     asset.href,
#     storage_options=asset.extra_fields["xarray:storage_options"],
#     **asset.extra_fields["xarray:open_kwargs"]
# )

# ----------------------------
# Define Variables & Windows
# ----------------------------
variables = [
    "air_temperature",
    "precipitation_amount",
    "vapor_pressure_deficit",
    "surface_downwelling_shortwave_flux_in_air",
    "wind_speed",
    "wind_from_direction",
    "relative_humidity",
    "specific_humidity",
    "potential_evapotranspiration",
    "burning_index_g",
    "dead_fuel_moisture_100hr",
    "dead_fuel_moisture_1000hr"
]

windows = [7, 30]  # days before observation

# ----------------------------
# Feature Extraction Function
# ----------------------------
def extract_features(row, ds, variables, windows):
    lat, lon, date = row["decimalLatitude"], row["decimalLongitude"], row["datetime"]
    features = {}

    for var in variables:
        for window in windows:
            start = date - timedelta(days=window)
            end = date

            try:
                chunk = ds[var].sel(time=slice(start, end)).interp(lat=lat, lon=lon)
                value = chunk.sum(dim="time") if "precip" in var else chunk.mean(dim="time")
                features[f"{var}_{window}d"] = value.compute().item()
            except Exception:
                features[f"{var}_{window}d"] = None

    return pd.Series(features)

# ----------------------------
# Dask Parallel Execution
# ----------------------------
tasks = [delayed(extract_features)(row, ds, variables, windows) for _, row in df.iterrows()]
results = compute(*tasks)

# ----------------------------
# Combine with GBIF Data
# ----------------------------
df_features = pd.concat([df.reset_index(drop=True), pd.DataFrame(results)], axis=1)
df_features.to_csv("gbif_with_all_gridmet_features.csv", index=False)

## TESTING

In [23]:
final = test.copy()
final.head(3)

display(int(final['reproductiveCondition'].value_counts().values.sum()) / final.shape[0] * 100)
display(int(final['elevation'].value_counts().values.sum()) / final.shape[0] * 100)

8.886036228783341

6.4898017401226635

In [8]:
import pandas as pd
from datetime import timedelta
import xarray as xr
import pystac_client
import planetary_computer

# ----------------------------
# Load GBIF cleaned dataset
# ----------------------------
df = pd.read_csv("../data/processed/date_location_occurrances.csv")
df["datetime"] = pd.to_datetime(df["datetime"])

# ----------------------------
# Filter to contiguous US lat/lon bounds
# ----------------------------
df = df[
    (df["decimalLatitude"] >= 25) & (df["decimalLatitude"] <= 49.5) &
    (df["decimalLongitude"] >= -125) & (df["decimalLongitude"] <= -66)
]

# Sample 5 rows for testing
sample_df = df.sample(5, random_state=42).reset_index(drop=True)

# ----------------------------
# Load GridMET dataset
# ----------------------------
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

asset = catalog.get_collection("gridmet").assets["zarr-abfs"]
ds = xr.open_zarr(
    asset.href,
    storage_options=asset.extra_fields["xarray:storage_options"],
    **asset.extra_fields["xarray:open_kwargs"]
)

# ----------------------------
# Variables and Time Windows
# ----------------------------
variables = [
    "air_temperature",
    "precipitation_amount",
    "mean_vapor_pressure_deficit",  # ✅ fixed name
    "surface_downwelling_shortwave_flux_in_air",
    "wind_speed",
    "wind_from_direction",
    "relative_humidity",
    "specific_humidity",
    "potential_evapotranspiration",
    "burning_index_g",
    "dead_fuel_moisture_100hr",
    "dead_fuel_moisture_1000hr"
]

windows = [7, 30]

# ----------------------------
# Feature Extraction Loop
# ----------------------------
features_list = []

for i, row in sample_df.iterrows():
    lat, lon, date = row["decimalLatitude"], row["decimalLongitude"], row["datetime"]
    feats = {}
    
    for var in variables:
        for win in windows:
            start = date - timedelta(days=win)
            end = date

            try:
                sel = ds[var].sel(time=slice(start, end)).interp(lat=lat, lon=lon)
                val = sel.sum(dim="time") if "precip" in var else sel.mean(dim="time")
                feats[f"{var}_{win}d"] = float(val.values)
            except Exception as e:
                print(f"⚠️ {var} failed at ({lat}, {lon}) on {date.date()} → {e}")
                feats[f"{var}_{win}d"] = None

    features_list.append(feats)

# ----------------------------
# Save merged result
# ----------------------------
features_df = pd.concat([sample_df, pd.DataFrame(features_list)], axis=1)
features_df.to_csv("test_gridmet_features_filtered.csv", index=False)

print("✅ Extraction complete. Saved to 'test_gridmet_features_filtered.csv'")

✅ Extraction complete. Saved to 'test_gridmet_features_filtered.csv'


In [2]:
import pandas as pd
import numpy as np
import dask.array as da
from datetime import timedelta
from tqdm import tqdm

def extract_gridmet_features(df, ds, window_days=(7, 30), variables=None):
    if variables is None:
        variables = [
            "air_temperature", "precipitation_amount", "mean_vapor_pressure_deficit",
            "surface_downwelling_shortwave_flux_in_air", "wind_speed", "wind_from_direction",
            "relative_humidity", "specific_humidity", "potential_evapotranspiration",
            "burning_index_g", "dead_fuel_moisture_100hr", "dead_fuel_moisture_1000hr"
        ]

    output = []

    # Convert string dates to datetime if needed
    df['datetime'] = pd.to_datetime(df['datetime'])

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Extracting weather features"):
        lat, lon, date = row["decimalLatitude"], row["decimalLongitude"], row["datetime"]
        feature_row = row.to_dict()

        for var in variables:
            for win in window_days:
                try:
                    time_start = np.datetime64(date - timedelta(days=win))
                    time_end = np.datetime64(date)

                    # Extract the sub-data for this variable and time window
                    var_data = ds[var].sel(
                        lat=lat, lon=lon, method="nearest"
                    ).sel(time=slice(time_start, time_end))

                    # Compute the mean value over that window
                    mean_val = var_data.mean().compute()
                    feature_row[f"{var}_{win}d"] = float(mean_val)

                except Exception as e:
                    feature_row[f"{var}_{win}d"] = np.nan
                    print(f"⚠️ {var} failed at ({lat}, {lon}) on {date.date()} → {e}")

        output.append(feature_row)

    return pd.DataFrame(output)

In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import pystac_client
import planetary_computer
from tqdm import tqdm
import time
import os
from datetime import datetime

# Load GridMET dataset
def load_gridmet_dataset():
    print("🔄 Loading fresh GridMET Zarr dataset from Planetary Computer...")
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=planetary_computer.sign_inplace,
    )
    asset = catalog.get_collection("gridmet").assets["zarr-abfs"]
    return xr.open_zarr(
        asset.href,
        storage_options=asset.extra_fields["xarray:storage_options"],
        **asset.extra_fields["xarray:open_kwargs"]
    )

# Define output file
output_file = "huckleberry_rowwise_output.csv"

# Check for header
if not os.path.exists(output_file):
    with open(output_file, "w") as f:
        header = [
            "gbifID", "datetime", "decimalLatitude", "decimalLongitude"
        ] + [f"{var}_{d}" for var in [
            "air_temperature", "precipitation_amount", "specific_humidity",
            "surface_downwelling_shortwave_flux_in_air", "wind_speed",
            "wind_from_direction", "relative_humidity", "potential_evapotranspiration",
            "burning_index_g", "dead_fuel_moisture_100hr", "dead_fuel_moisture_1000hr"
        ] for d in ["7d", "30d"]]
        f.write(",".join(header) + "\n")

# Load dataset and input
ds = load_gridmet_dataset()
df = pd.read_csv("../data/processed/date_location_occurrances.csv")

# Process each row
with open(output_file, "a", buffering=1) as f:
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        lat, lon = row["decimalLatitude"], row["decimalLongitude"]
        try:
            date = pd.to_datetime(row["datetime"])
        except:
            print(f"⚠️ Skipping invalid date at row {idx}")
            continue

        # Filter: GridMET range (lat: 25–50, lon: -125 to -104) & after 1979
        if not (25 <= lat <= 50 and -125 <= lon <= -104 and date.year >= 1980):
            continue

        entry = {
            "gbifID": row["gbifID"],
            "datetime": date,
            "decimalLatitude": lat,
            "decimalLongitude": lon,
        }

        for attempt in range(2):
            try:
                for var in [
                    "air_temperature", "precipitation_amount", "specific_humidity",
                    "surface_downwelling_shortwave_flux_in_air", "wind_speed",
                    "wind_from_direction", "relative_humidity", "potential_evapotranspiration",
                    "burning_index_g", "dead_fuel_moisture_100hr", "dead_fuel_moisture_1000hr"
                ]:
                    for delta in [7, 30]:
                        val = ds[var].sel(
                            time=date - pd.Timedelta(days=delta),
                            lat=lat, lon=lon, method="nearest"
                        ).values.item()
                        entry[f"{var}_{delta}d"] = val
                break  # success
            except Exception as e:
                if "Signature not valid in the specified time frame" in str(e) and attempt == 0:
                    print(f"🔁 Signature expired for row {idx} on {date.date()}, reloading...")
                    ds = load_gridmet_dataset()
                    time.sleep(1)
                else:
                    print(f"⚠️ Failed at row {idx} ({lat}, {lon}) on {date.date()}: {e}")
                    for var in [
                        "air_temperature", "precipitation_amount", "specific_humidity",
                        "surface_downwelling_shortwave_flux_in_air", "wind_speed",
                        "wind_from_direction", "relative_humidity", "potential_evapotranspiration",
                        "burning_index_g", "dead_fuel_moisture_100hr", "dead_fuel_moisture_1000hr"
                    ]:
                        entry[f"{var}_7d"] = None
                        entry[f"{var}_30d"] = None
                    break

        # Write to CSV
        line = ",".join(str(entry[k]) for k in header)
        f.write(line + "\n")
        f.flush()  # ensure it writes immediately
        print(f"✅ Wrote row {idx} to CSV")

🔄 Loading fresh GridMET Zarr dataset from Planetary Computer...


  0%|          | 18/6275 [00:34<3:21:35,  1.93s/it]

✅ Wrote row 17 to CSV


  0%|          | 19/6275 [00:49<4:57:26,  2.85s/it]

✅ Wrote row 18 to CSV


...

✅ Wrote row 958 to CSV


 15%|█▌        | 960/6275 [3:54:16<32:20:47, 21.91s/it]

✅ Wrote row 959 to CSV


 15%|█▌        | 960/6275 [3:54:32<21:38:30, 14.66s/it]


KeyboardInterrupt: 